# Evaluate Falcon 40B for summarization with CNN Daily Mail and ROUGE
**Please note, this is a modified version of the JumpStart notebook for Falcon 40B. Specifically we add the section on evaluation with ROUGE, in addition to the boto3 invocation**

---
In this demo notebook, we demonstrate how to use the SageMaker Python SDK to deploy Falcon models for text generation. It is a permissively licensed ([Apache-2.0](https://jumpstart-cache-prod-us-east-2.s3.us-east-2.amazonaws.com/licenses/Apache-License/LICENSE-2.0.txt)) open source model trained on the [RefinedWeb dataset](https://huggingface.co/datasets/tiiuae/falcon-refinedweb). We show several example use cases including code generation, question answering, translation etc.

---

In [8]:
!pip install sagemaker --quiet --upgrade --force-reinstall
!pip install ipywidgets==7.0.0 --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.3.0 requires daal==2021.2.3, which is not installed.
spyder 5.1.5 requires pyqt5<5.13, which is not installed.
spyder 5.1.5 requires pyqtwebengine<5.13, which is not installed.
autovizwidget 0.20.5 requires pandas<2.0.0,>=0.20.1, but you have pandas 2.0.3 which is incompatible.
awscli 1.27.153 requires botocore==1.29.153, but you have botocore 1.31.2 which is incompatible.
awscli 1.27.153 requires PyYAML<5.5,>=3.10, but you have pyyaml 6.0 which is incompatible.
docker-compose 1.29.2 requires jsonschema<4,>=2.5.1, but you have jsonschema 4.18.0 which is incompatible.
docker-compose 1.29.2 requires PyYAML<6,>=3.10, but you have pyyaml 6.0 which is incompatible.
hdijupyterutils 0.20.5 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.0.3 which is incompatible.
jupyterlab 3.2.1 requires jupyter-serve

In [3]:
model_id, model_version, = (
    "huggingface-llm-falcon-40b-instruct-bf16",
    "*",
)

In [4]:
from ipywidgets import Dropdown

model_ids = ['huggingface-llm-falcon-40b-bf16',
             'huggingface-llm-falcon-40b-instruct-bf16',
             'huggingface-llm-falcon-7b-bf16',
             'huggingface-llm-falcon-7b-instruct-bf16']

# display the model-ids in a dropdown to select a model for inference.
model_dropdown = Dropdown(
    options=model_ids,
    value=model_id,
    description="Select a model",
    style={"description_width": "initial"},
    layout={"width": "max-content"},
)
display(model_dropdown)

A Jupyter Widget

In [5]:
model_id = model_dropdown.value

In [146]:
%%time
from sagemaker.jumpstart.model import JumpStartModel

my_model = JumpStartModel(model_id=model_id)
predictor = my_model.deploy()

### Changing instance type
---


Models have been tested on the following instance types:

 - Falcon 7B and 7B instruct: `ml.g5.2xlarge`, `ml.g5.2xlarge`, `ml.g5.4xlarge`, `ml.g5.8xlarge`, `ml.g5.16xlarge`, `ml.g5.12xlarge`, `ml.g5.24xlarge`, `ml.g5.48xlarge`, `ml.p4d.24xlarge`
 - Falcon 40B and 40B instruct: `ml.g5.12xlarge`, `ml.g5.48xlarge`, `ml.p4d.24xlarge`

If an instance type is not available in you region, please try a different instance. You can do so by specifying instance type in the JumpStartModel class.

`my_model = JumpStartModel(model_id="huggingface-llm-falcon-40b-instruct-bf16", instance_type="ml.g5.12xlarge")`

---

### Changing number of GPUs
---
Falcon models are served with HuggingFace (HF) LLM DLC which requires specifying number of GPUs during model deployment. 

**Falcon 7B and 7B instruct:** HF LLM DLC currently does not support sharding for 7B model. Thus, even if more than one GPU is available on the instance, please do not increase number of GPUs. 

**Falcon 40B and 40B instruct:** By default number of GPUs are set to 4. However, if you are using `ml.g5.48xlarge` or `ml.p4d.24xlarge`, you can increase number of GPUs to be 8 as follows: 

`my_model = JumpStartModel(model_id="huggingface-llm-falcon-40b-instruct-bf16", instance_type="ml.g5.48xlarge")`

`my_model.env['SM_NUM_GPUS'] = '8'`

`predictor = my_model.deploy()`


---

In [166]:
import boto3

client = boto3.client('sagemaker-runtime')

In [183]:
# if you are in the same Jupyter session with a defined predictor object, you use
# predictor.predict(payload) in the SageMaker Python SDK
# otherwise if you are using a precreated endpoint, you can the endpoint name with boto3 
endpoint_name = 'hf-llm-falcon-40b-instruct-bf16-2023-07-10-15-19-42-754'

In [208]:
%%time
import json

data = {
    "inputs": "What is the purpose of life?",
    "properties": {
        "min_length": 100,
        "max_length": 150,
        "do_sample": True,
    },
}

response_model = client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps(data),
    ContentType="application/json",
)

response_model["Body"].read().decode("utf8")

CPU times: user 14 ms, sys: 2.67 ms, total: 16.7 ms
Wall time: 1.09 s


'[{"generated_text":"\\nAs an AI language model, I do not have a personal belief system or a purpose in life"}]'

In [169]:
%%time


prompt = "Tell me about Amazon SageMaker."

payload = {
    "inputs": prompt,
    "parameters": {
        "do_sample": True,
        "top_p": 0.9,
        "temperature": 0.8,
        "max_new_tokens": 1024,
        "stop": ["<|endoftext|>", "</s>"]
    }
}

response_model = client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps(payload),
    ContentType="application/json",
)

response_model["Body"].read().decode("utf8")

CPU times: user 1.11 ms, sys: 2.86 ms, total: 3.97 ms
Wall time: 5.93 s


'[{"generated_text":"\\nAmazon SageMaker is a fully managed platform for building, training, and deploying machine learning models. It allows data scientists and developers to quickly create and deploy machine learning models using the tools of their choice. SageMaker provides a set of pre-built algorithms, as well as the ability to build custom models using popular tools such as TensorFlow, Apache MXNet, and Scikit-learn. SageMaker also provides the ability to manage and deploy models in the cloud, as well as the ability to continuously monitor and retrain models based on new data."}]'

### About the model

---
Falcon is a causal decoder-only model built by [Technology Innovation Institute](https://www.tii.ae/) (TII) and trained on more than 1 trillion tokens of RefinedWeb enhanced with curated corpora. It was built using custom-built tooling for data pre-processing and model training built on Amazon SageMaker. As of June 6, 2023, it is the best open-source model currently available. Falcon-40B outperforms LLaMA, StableLM, RedPajama, MPT, etc. To see comparison, see [OpenLLM Leaderboard](https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard). It features an architecture optimized for inference, with FlashAttention and multiquery. 


[Refined Web Dataset](https://huggingface.co/datasets/tiiuae/falcon-refinedweb): Falcon RefinedWeb is a massive English web dataset built by TII and released under an Apache 2.0 license. It is a highly filtered dataset with large scale de-duplication of CommonCrawl. It is observed that models trained on RefinedWeb achieve performance equal to or better than performance achieved by training model on curated datasets, while only relying on web data.

**Model Sizes:**
- **Falcon-7b**: It is a 7 billion parameter model trained on 1.5 trillion tokens. It outperforms comparable open-source models (e.g., MPT-7B, StableLM, RedPajama etc.). To see comparison, see [OpenLLM Leaderboard](https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard). To use this model, please select `model_id` in the cell above to be "huggingface-textgeneration-falcon-7b-bf16".
- **Falcon-40B**: It is a 40 billion parameter model trained on 1 trillion tokens.  It has surpassed renowned models like LLaMA-65B, StableLM, RedPajama and MPT on the public leaderboard maintained by Hugging Face, demonstrating its exceptional performance without specialized fine-tuning. To see comparison, see [OpenLLM Leaderboard](https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard). 

**Instruct models (Falcon-7b-instruct/Falcon-40B-instruct):** Instruct models are base falcon models fine-tuned on a mixture of chat and instruction datasets. They are ready-to-use chat/instruct models.  To use these models, please select `model_id` in the cell above to be "huggingface-textgeneration-falcon-7b-instruct-bf16" or "huggingface-textgeneration-falcon-40b-instruct-bf16".

It is [recommended](https://huggingface.co/tiiuae/falcon-7b) that Instruct models should be used without fine-tuning and base models should be fine-tuned further on the specific task.

**Limitations:**

- Falcon models are mostly trained on English data and may not generalize to other languages. 
- Falcon carries the stereotypes and biases commonly encountered online and in the training data. Hence, it is recommended to develop guardrails and to take appropriate precautions for any production use. This is a raw, pretrained model, which should be further finetuned for most usecases.


---

In [170]:
def query_endpoint(payload, endpoint_name = 'hf-llm-falcon-40b-instruct-bf16-2023-07-10-15-19-42-754'):
    
    response_model = client.invoke_endpoint(
        EndpointName=endpoint_name,
        Body=json.dumps(payload),
        ContentType="application/json",
    )

    res = response_model["Body"].read().decode("utf8")
    
    out = json.loads(res)
    
    return out[0]['generated_text']

In [209]:
payload = {
    "inputs": "Building a website can be done in 10 simple steps:",
    "parameters":{
        "max_new_tokens": 300,
        "no_repeat_ngram_size": 3
        }
}
print(query_endpoint(payload))



1. Choose a domain name and hosting provider.
2. Install a content management system (CMS) like WordPress.
3. Choose a theme and customize it to your liking.
4. Create pages and add content.
5. Optimize your website for search engines.
6. Test your website for usability and functionality.
7. Launch your website and promote it.
8. Monitor your website's performance and make improvements.
9. Maintain your website by updating it regularly.
10. Continuously improve your website to keep it relevant and engaging.


In [178]:
# Translation
payload = {
    "inputs": """Translate English to French:

    sea otter => loutre de mer

    peppermint => menthe poivrée

    plush girafe => girafe peluche

    cheese =>""",
    "parameters":{
        "max_new_tokens": 3
    }
}



print(query_endpoint(payload))

 fromage

   


In [180]:
# Sentiment-analysis
payload = {
    "inputs": """"I hate it when my phone battery dies."
                Sentiment: Negative
                ###
                Tweet: "My day has been :+1:"
                Sentiment: Positive
                ###
                Tweet: "This is the link to the article"
                Sentiment: Neutral
                ###
                Tweet: "This new music video was incredibile"
                Sentiment:""",
    "parameters": {
        "max_new_tokens":2
    }
}
print(query_endpoint(payload))

 Positive
                


In [181]:
# Question answering
payload = {
    "inputs": "Could you remind me when was the C programming language invented?",
    "parameters":{
        "max_new_tokens": 50
    }
}
print(query_endpoint(payload))


The C programming language was invented in the early 1970s by Dennis Ritchie at Bell Labs.


In [182]:
# Recipe generation
payload = {"inputs": "What is the recipe for a delicious lemon cheesecake?", "parameters":{"max_new_tokens": 400}}
print(query_endpoint(payload))


Ingredients:
- 1 1/2 cups graham cracker crumbs
- 1/4 cup granulated sugar
- 1/3 cup unsalted butter, melted
- 3 (8-ounce) packages cream cheese, softened
- 1 1/2 cups granulated sugar
- 1/4 cup all-purpose flour
- 1/2 cup sour cream
- 1/2 cup heavy cream
- 1 tablespoon lemon zest
- 1/4 cup fresh lemon juice
- 4 large eggs

Instructions:
1. Preheat oven to 325°F.
2. In a medium bowl, combine graham cracker crumbs, sugar, and melted butter. Press mixture into the bottom of a 9-inch springform pan.
3. In a large mixing bowl, beat cream cheese until smooth. Add sugar and flour, and beat until well combined.
4. Add sour cream, heavy cream, lemon zest, and lemon juice, and beat until smooth.
5. Add eggs one at a time, beating well after each addition.
6. Pour mixture into prepared crust.
7. Bake for 1 hour and 15 minutes, or until center is set.
8. Allow cheesecake to cool completely before serving.


In [184]:
# Summarization

payload = {
    "inputs":"""Starting today, the state-of-the-art Falcon 40B foundation model from Technology
    Innovation Institute (TII) is available on Amazon SageMaker JumpStart, SageMaker's machine learning (ML) hub
    that offers pre-trained models, built-in algorithms, and pre-built solution templates to help you quickly get
    started with ML. You can deploy and use this Falcon LLM with a few clicks in SageMaker Studio or
    programmatically through the SageMaker Python SDK.
    Falcon 40B is a 40-billion-parameter large language model (LLM) available under the Apache 2.0 license that
    ranked #1 in Hugging Face Open LLM leaderboard, which tracks, ranks, and evaluates LLMs across multiple
    benchmarks to identify top performing models. Since its release in May 2023, Falcon 40B has demonstrated
    exceptional performance without specialized fine-tuning. To make it easier for customers to access this
    state-of-the-art model, AWS has made Falcon 40B available to customers via Amazon SageMaker JumpStart.
    Now customers can quickly and easily deploy their own Falcon 40B model and customize it to fit their specific
    needs for applications such as translation, question answering, and summarizing information.
    Falcon 40B are generally available today through Amazon SageMaker JumpStart in US East (Ohio),
    US East (N. Virginia), US West (Oregon), Asia Pacific (Tokyo), Asia Pacific (Seoul), Asia Pacific (Mumbai),
    Europe (London), Europe (Frankfurt), Europe (Ireland), and Canada (Central),
    with availability in additional AWS Regions coming soon. To learn how to use this new feature,
    please see SageMaker JumpStart documentation, the Introduction to SageMaker JumpStart –
    Text Generation with Falcon LLMs example notebook, and the blog Technology Innovation Institute trainsthe
    state-of-the-art Falcon LLM 40B foundation model on Amazon SageMaker. Summarize the article above:""",
    "parameters":{
        "max_new_tokens":60
        }
    }
print(query_endpoint(payload))


    Amazon SageMaker JumpStart now offers the state-of-the-art Falcon 40B foundation model from Technology
    Innovation Institute (TII) as a pre-trained model. Customers can quickly and easily deploy their own Falcon 40B
    model and customize it to fit their specific


# Evaluate the LLM for summarization performance with CNN Dailymail and ROUGE

In [7]:
%%writefile requirements.txt
torch
transformers
datasets
evaluate
rouge_score
absl-py

Overwriting requirements.txt


In [185]:
!pip install -r requirements.txt

In [186]:
from datasets import load_dataset

dataset = load_dataset("cnn_dailymail", '3.0.0')

A Jupyter Widget

In [187]:
dataset

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [213]:
article_id = 1
article = dataset['train'][article_id]['article']

In [215]:
article

'Editor\'s note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O\'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor." Here, inmates with the most severe mental illnesses are incarcerated until they\'re ready to appear in court. Most often, they face drug charges or charges of assaulting an officer --charges that Judge Steven Leifman says are usually "avoidable felonies." He says the arrests often result from confrontations with police. Mentally ill people often won\'t do what they\'re told when police arrive on the scene -- confrontation seems to exacerbate their illness and they become more paranoid, delusional, and less likely to foll

In [216]:
# Summarization

char_cutoff = 2000

payload = {
    # we'll skip articles longer than the cutoff in the eval loop
    "inputs":f'{article[:char_cutoff]}  Highlights:',
    "parameters":{
        "max_new_tokens":66
        }
    }
predictions = query_endpoint(payload)

In [217]:
print(predictions)

 • Judge Steven Leifman says mentally ill people often won't do what they're told when police arrive on the scene • Mentally ill people often end up on the ninth floor severely mentally disturbed, but not getting any real help because they're in jail • Leifman says about one-third


In [218]:
# these are not objective summaries. they are news highlights - picked specifically to optimize for clicks and making a story
dataset['train'][article_id]['highlights']

'Mentally ill inmates in Miami are housed on the "forgotten floor"\nJudge Steven Leifman says most are there as a result of "avoidable felonies"\nWhile CNN tours facility, patient shouts: "I am the son of the president"\nLeifman says the system is unjust and he\'s fighting for change .'

In [219]:
import evaluate

rouge = evaluate.load('rouge')

In [220]:
def equalize_rouge_inputs(predictions, label):
    if len(predictions) < len(label):
        label = label[:len(predictions)]

    elif len(label) < len(predictions):
        predictions = predictions[:len(label)]

    assert len(label) == len(predictions)
    
    return predictions, label

In [221]:
predictions, label = equalize_rouge_inputs(predictions, label)

results = rouge.compute(predictions=predictions,
                         references=label)

rouge1 = results['rouge1']

In [222]:
# 1 for rouge1 means the result is perfect
results['rouge1']

0.024242424242424242

In [207]:
import time

num_samples = 10

for idx in range(num_samples):
    
    article = dataset['train'][idx]['article'][:char_cutoff]
    
    label = dataset['train'][idx]['highlights']
    
    payload = { "inputs":f'{article}  Highlights:',
                "parameters":{
                    # may want to parameterize this by the highlight length as well
                    "max_new_tokens":66
                    }
                }
    
    predictions = query_endpoint(payload)
    
    predictions, label = equalize_rouge_inputs(predictions, label)

    results = rouge.compute(predictions=predictions,
                             references=label)

    rouge1 = results['rouge1']

    with open('summarization_results.txt', 'a') as f:
        f.write(f'=========== Article {idx}====== \n')
        f.write(predictions)
        f.write(f' \n ROUGE1: {rouge1} \n')
            
    time.sleep(1)

### Supported parameters

***
Some of the supported parameters while performing inference are the following:

* **max_length:** Model generates text until the output length (which includes the input context length) reaches `max_length`. If specified, it must be a positive integer.
* **max_new_tokens:** Model generates text until the output length (excluding the input context length) reaches `max_new_tokens`. If specified, it must be a positive integer.
* **num_beams:** Number of beams used in the greedy search. If specified, it must be integer greater than or equal to `num_return_sequences`.
* **no_repeat_ngram_size:** Model ensures that a sequence of words of `no_repeat_ngram_size` is not repeated in the output sequence. If specified, it must be a positive integer greater than 1.
* **temperature:** Controls the randomness in the output. Higher temperature results in output sequence with low-probability words and lower temperature results in output sequence with high-probability words. If `temperature` -> 0, it results in greedy decoding. If specified, it must be a positive float.
* **early_stopping:** If True, text generation is finished when all beam hypotheses reach the end of sentence token. If specified, it must be boolean.
* **do_sample:** If True, sample the next word as per the likelihood. If specified, it must be boolean.
* **top_k:** In each step of text generation, sample from only the `top_k` most likely words. If specified, it must be a positive integer.
* **top_p:** In each step of text generation, sample from the smallest possible set of words with cumulative probability `top_p`. If specified, it must be a float between 0 and 1.
* **return_full_text:** If True, input text will be part of the output generated text. If specified, it must be boolean. The default value for it is False.
* **stop**: If specified, it must a list of strings. Text generation stops if any one of the specified strings is generated.

We may specify any subset of the parameters mentioned above while invoking an endpoint. 

For more parameters and information on HF LLM DLC, please see [this article](https://huggingface.co/blog/sagemaker-huggingface-llm#4-run-inference-and-chat-with-our-model).
***

### 5. Clean up the endpoint

In [ ]:
# Delete the SageMaker endpoint
predictor.delete_model()
predictor.delete_endpoint()